In [65]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import meshio
import numpy as np

class VesselDataset(Dataset):

    def __init__(self, data_path, num_points=10_000, seed=None):
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)
        else:
            torch.seed()
            np.random.seed()
            
        self.data_path = data_path
        self.num_points = num_points
        self.vessel_files = self._get_vessel_files()
    

    def __len__(self):
        return len(self.vessel_files)

    def __getitem__(self, idx): 
        raw_data = self._point_data_generator(self.vessel_files[idx])
        transformed_data = self._transform_data(raw_data)
        return transformed_data
    
    def _get_vessel_files(self):
        """
        Retrieves the vessel files from the given directory.

        Returns:
        - list: A list of dictionaries, where each dictionary contains the paths to the fluid and mesh files.
        """
        vessel_files = []
        for fluid_filename in os.listdir(self.data_path):
            if fluid_filename.endswith('fluid.vtu'):
                fluid_file_path = os.path.join(self.data_path, fluid_filename)
                mesh_file_path = fluid_file_path.replace('fluid.vtu', 'wss.vtu')
                vessel_files.append({
                    'fluid': fluid_file_path,
                    'mesh': mesh_file_path
                })
        vessel_files = np.array(vessel_files)
        return vessel_files
        
    def _point_data_generator(self, vessel_file:dict):
        mesh_data = meshio.read(vessel_file['mesh'])
        fluid_data = meshio.read(vessel_file['fluid'])
        idx = np.random.choice(np.arange(fluid_data.points.shape[0]), self.num_points, replace=False)
        return (
            torch.concat([torch.Tensor(fluid_data.points), torch.Tensor(mesh_data.points)], axis=0)[idx],
            torch.concat([torch.Tensor(fluid_data.point_data['velocity_systolic']), torch.zeros(mesh_data.points.shape)], axis=0)[idx]
        )
        
    def _transform_data(self, raw_data):
        points, velocities = raw_data
        # Normalize points
        points_min = points.min(dim=0, keepdim=True)[0]
        points_max = points.max(dim=0, keepdim=True)[0]
        points = (points - points_min) / (points_max - points_min)
        
        # Normalize velocities
        velocities_min = velocities.min(dim=0, keepdim=True)[0]
        velocities_max = velocities.max(dim=0, keepdim=True)[0]
        velocities = (velocities - velocities_min) / (velocities_max - velocities_min)
        
        transformed_data = (points, velocities)
        return transformed_data
        

In [66]:
from utils import helper_functions as hf

DATA_DIR = hf.get_project_root() / "data" / "carotid_flow_database"

dataset = VesselDataset(DATA_DIR, seed=666, num_points=2**14)


In [67]:
dataset[0]

(tensor([[0.5977, 0.9025, 0.9175],
         [0.7780, 0.4754, 0.5630],
         [0.7306, 0.7770, 0.9250],
         ...,
         [0.5283, 0.8938, 0.9875],
         [0.7848, 0.8543, 0.9215],
         [0.6514, 0.4937, 0.6631]]),
 tensor([[0.8312, 0.5283, 0.2789],
         [0.7661, 0.5935, 0.5132],
         [0.8350, 0.5253, 0.2791],
         ...,
         [0.8312, 0.5283, 0.2789],
         [0.8141, 0.6180, 0.2705],
         [0.6939, 0.4990, 0.2858]]))

In [72]:
points_len = []
for v in dataset.vessel_files:
    #print(v)
    mesh_data = meshio.read(v['mesh'])
    fluid_data = meshio.read(v['fluid'])
    points = torch.concat([torch.Tensor(fluid_data.points), torch.Tensor(mesh_data.points)], axis=0)
    points_len.append(points.shape[0])

points_len = np.array(points_len)
print(np.min(points_len))
print(np.max(points_len))
print(np.mean(points_len))
print(np.median(points_len))
print(np.std(points_len))
print(np.percentile(points_len, 25))
print(np.percentile(points_len, 75))
print(np.percentile(points_len, 90))
print(np.percentile(points_len, 95))

90674
408819
221205.30534351146
213490.0
70369.29526899397
175143.5
265617.0
310551.0
359799.5


In [68]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=2)
for batch in dataloader:
    print(batch)
    break

[tensor([[[0.1501, 0.8338, 0.6671],
         [0.3341, 0.9784, 0.8220],
         [0.1431, 0.6293, 0.9405],
         ...,
         [0.2650, 0.7755, 0.9091],
         [0.2290, 0.7263, 0.6795],
         [0.2646, 0.6925, 0.8441]],

        [[0.3469, 0.3809, 0.8417],
         [0.9771, 0.2466, 0.0834],
         [0.4456, 0.6446, 0.7740],
         ...,
         [0.5564, 0.6506, 0.6629],
         [0.5361, 0.4245, 0.9789],
         [0.5367, 0.4233, 0.9777]]]), tensor([[[0.6688, 0.3861, 0.2942],
         [0.9922, 0.5380, 0.5451],
         [0.8435, 0.3575, 0.4657],
         ...,
         [0.7357, 0.3779, 0.3655],
         [0.7123, 0.4110, 0.3667],
         [0.6973, 0.4278, 0.4220]],

        [[0.5838, 0.4899, 0.3784],
         [0.5838, 0.4899, 0.3784],
         [0.5838, 0.4899, 0.3784],
         ...,
         [0.5749, 0.5032, 0.4033],
         [0.5845, 0.5677, 0.5663],
         [0.5841, 0.5599, 0.5507]]])]


In [60]:
b1, b2 = batch

In [61]:
b1.shape, b2.shape

(torch.Size([2, 16384, 3]), torch.Size([2, 16384, 3]))

In [37]:
batch[0].shape, batch[1].shape

(torch.Size([2, 16384, 3]), torch.Size([2, 16384, 3]))

In [45]:
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(2**14*3, 2**12),
    nn.ReLU(),
    nn.Linear(2**12, 2**10),
    nn.ReLU(),
    nn.Linear(2**10, 2**12),
    nn.ReLU(),
    nn.Linear(2**12, 2**14*3),
)

In [55]:
out = model(batch[0])
out.shape

torch.Size([2, 49152])

In [56]:
out = out.reshape(batch[1].shape)
out.shape

torch.Size([2, 16384, 3])

In [57]:
criterion = nn.MSELoss()
loss = criterion(out, batch[1])
loss

tensor(2.7733, grad_fn=<MseLossBackward0>)